<a href="https://colab.research.google.com/github/putriayualc/Machine-Learning/blob/main/Jobsheet-10/Praktikum%202%20dan%20Tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Putri Ayu Aliciawati  
Kelas : TI-3C  
Absen : 21  
NIM : 2241720132

# Praktikum 2

## Setup

### Import TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

### Download Dataset Shakespeare

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


### Load Data

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Olah Teks

### Vectorize Teks

In [7]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Prediksi

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [18]:
seq_length = 100

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [21]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Membuat Batch Training

In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Buat Model

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Uji Model

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [33]:
sampled_indices

array([24, 16, 21, 31, 13, 49, 10, 18, 31,  2,  6, 33, 22, 48, 17,  4, 20,
       34, 59, 49, 55, 35, 43, 37, 25,  8, 64, 58, 64, 56,  7, 21, 19, 18,
       16,  0, 19, 23, 59,  0, 18, 52, 11, 63, 13, 18, 41, 10, 34, 41, 56,
       33, 41, 23, 14, 59, 24, 44, 52, 61, 15,  9, 50, 54, 58, 44,  3, 30,
        9, 21, 57, 23, 53, 37, 24, 30, 64,  7, 40, 57, 26,  8, 64, 20, 36,
       43, 60, 65, 12, 46, 13, 38, 19, 10,  5,  4, 32, 64, 41, 11])

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"me one of you would fly from us,\nThat there's no hoped-for mercy with the brothers\nMore than with ru"

Next Char Predictions:
 b"KCHR?j3ER 'TIiD$GUtjpVdXL-ysyq,HFEC[UNK]FJt[UNK]Em:x?Eb3UbqTbJAtKemvB.kose!Q.HrJnXKQy,arM-yGWduz;g?YF3&$Syb:"


## Train Model
### Tambahan optimizer dan fungsi loss

In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1890025, shape=(), dtype=float32)


In [37]:
tf.exp(example_batch_mean_loss).numpy()

65.95697

In [38]:
model.compile(optimizer='adam', loss=loss)

### Konfigurasi Checkpoints

In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Lakukan Proses Training

In [42]:
EPOCHS = 20

In [43]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 124s 705ms/step - loss: 2.7326
Epoch 2/20
172/172 [==============================] - 122s 703ms/step - loss: 1.9943
Epoch 3/20
172/172 [==============================] - 118s 683ms/step - loss: 1.7108
Epoch 4/20
172/172 [==============================] - 125s 721ms/step - loss: 1.5480
Epoch 5/20
172/172 [==============================] - 123s 707ms/step - loss: 1.4493
Epoch 6/20
172/172 [==============================] - 117s 670ms/step - loss: 1.3824
Epoch 7/20
172/172 [==============================] - 117s 676ms/step - loss: 1.3301
Epoch 8/20
172/172 [==============================] - 118s 681ms/step - loss: 1.2859
Epoch 9/20
172/172 [==============================] - 120s 691ms/step - loss: 1.2446
Epoch 10/20
172/172 [==============================] - 118s 679ms/step - loss: 1.2064
Epoch 11/20
172/172 [==============================] - 119s 685ms/step - loss: 1.1664
Epoch 12/20
172/172 [==============================] - 116s 671

## Generate Teks

In [44]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [45]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [46]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
That when the sea?

ESCALUS:
The sought this man's woe, thou liest.
Thy lord again, or to fight with mildness.

PETRUCHIO:
To thee a king, without children; only to subject.

LUCENTIO:
Alack, for France, have ta'en the sea.

BRAKENBURY:
I knock him in his lady as my wife:
Since we are thou, tender-shriedding flowers of my own distress.

BRUTUS:
Come, shall I have.

BENVOLIO:
That's such a heavy moon, were ride's subst;
And who should say, 'Lover i' the right of your age:
I would they were to murder: mysteries
That thou art not a lady:' doth shew myself,
Take the owe unwillingness Liur dather.
And never had broke--the boy;
Of our cowards their own youth, if my referbrawn
your loves chastises, that was my wife.

Lord:
His name is Kater? 'Tis pardon 'em, deportuner;
Sometime she demissed upon him.

BUCKINGHAM:
No, sir, is it not kiss the people; for how mayst thou
love a woman:
Nuely to perform it from my head;
Young English marriage, we accomp out.

DUKE VINCENTIO:
Where is my son

In [47]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe lords of Rome; no doubt, son\nWas 'dward's; be made a goodly presence\nThan we finde, butt with a goodly base:\nProud-queen of patience to e'er come to me;\nUnder our that way thou art not hand of heaven;\nCould not over-me impartial; believe geath.\n\nVOLUMNIA:\nThe children yours upon him what let her intermand\nThat thou wert round'st to tread-she thunder?\n\nLUCIO:\nI cannot tell to this.\nFor live, I do, I wish't.\nMy love, our fourth, every trial of our highness\nTo be entirely death. O Ghoutster's, wife?\n\nPARIS:\nI got that in his grave speak; let me live,\nTo be your time to her! You'r boundy slew is a\ngreat palace bowlesh of his neck.\n\nANThIONE:\nGo to, against From From him\nDid this day read for and cureden: for, how age!\nWhat I was great, and more of this directing:\nit is much guilty oness, that hour I run?\nYou have pun her maid at once than death without thy frot?\n\nESCALUS:\nWhy, lady, as you were king?\n\nJULIET:\nWhy, how now, thou art

## Ekspor Model Generator

In [48]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [49]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The ratemble and the knees of the house. Go, jough, so are thou, good die:
I have no ngign of beats


# Tugas

In [50]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return {'loss': loss}

In [51]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [52]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [53]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 124s 703ms/step - loss: 2.7471


In [54]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
  start = time.time()

  mean.reset_states()
  for (batch_n, (inp, target)) in enumerate(dataset):
    logs = model.train_step([inp, target])
    mean.update_state(logs['loss'])
    if batch_n % 50 == 0:
      template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
      print(template)

        # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1984
Epoch 1 Batch 50 Loss 2.1015
Epoch 1 Batch 100 Loss 1.9839
Epoch 1 Batch 150 Loss 1.8940
Epoch 2 Batch 0 Loss 1.8384
Epoch 2 Batch 50 Loss 1.7903
Epoch 2 Batch 100 Loss 1.7129
Epoch 2 Batch 150 Loss 1.6467
Epoch 3 Batch 0 Loss 1.6081
Epoch 3 Batch 50 Loss 1.5867
Epoch 3 Batch 100 Loss 1.5076
Epoch 3 Batch 150 Loss 1.5083
Epoch 4 Batch 0 Loss 1.4027
Epoch 4 Batch 50 Loss 1.4794
Epoch 4 Batch 100 Loss 1.4540
Epoch 4 Batch 150 Loss 1.4367
Epoch 5 Batch 0 Loss 1.4599
Epoch 5 Batch 50 Loss 1.4112
Epoch 5 Batch 100 Loss 1.3956
Epoch 5 Batch 150 Loss 1.3781

Epoch 5 Loss: 1.3853
Time taken for 1 epoch 116.52 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 1.2751
Epoch 6 Batch 50 Loss 1.3263
Epoch 6 Batch 100 Loss 1.3208
Epoch 6 Batch 150 Loss 1.3314
Epoch 7 Batch 0 Loss 1.2078
Epoch 7 Batch 50 Loss 1.2861
Epoch 7 Batch 100 Loss 1.2653
Epoch 7 Batch 150 Loss 1.2686
Epoch 8 Batch 0 Loss 1.1958
Epoch 8 Batch 50

## Soal
sebutkan perbedaanya dengan praktikum 2?  
